# **Pandas_Functions**

In [11]:
import sys
sys.path.append("..")
from suntzu.test import Test
test = Test()
df = test.read_file("./Titanic.csv")

,sex,age,sibsp,parch,fare,embarked,class,who,alone,survived
0,male,22.0,1,0,7.2500,S,Third,man,False,0
1,female,38.0,1,0,71.2833,C,First,woman,False,1
2,female,26.0,0,0,7.9250,S,Third,woman,True,1
3,female,35.0,1,0,53.1000,S,First,woman,False,1
4,male,35.0,0,0,8.0500,S,Third,man,True,0


### **Reading Data**

In [7]:
from polars import read_csv, read_database, read_parquet, read_json
from xarray import open_dataset
from polars import DataFrame
def read_file(path):
    if path.endswith(".csv"):
        df = read_csv(path)
        df = df.to_pandas()
        return df

In [6]:
def set_full_view(max_cols="yes", max_rows="yes", max_width="yes", max_colwidht="yes"):
   from pandas import set_option
   if max_cols == "yes":
      set_option('display.max_rows', None)
   elif max_rows == "yes":
      set_option('display.max_columns', None)
   elif max_width == "yes":
      set_option('display.width', None)
   elif max_colwidht == "yes":
      set_option('display.max_colwidth', None)

Checking Libraries versions

Get Dtypes

Removing Cells

Get Memory Usage

Type Conversion

Verificar o joblib

Redefine Index

Drop Duplicates

Truncate Cells

Best Dtypes

Metadata

Renaming Axis

!jupyter nbextension enable --py widgetsnbextension

In [9]:
path = 'food_coded.csv'
df = read_file('food_coded.csv')
df.head()

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,cook,comfort_food_reasons_coded_duplicated_0,cuisine,diet_current,diet_current_coded,drink,eating_changes,eating_changes_coded,eating_changes_coded1,eating_out,employment,ethnic_food,exercise,father_education,father_profession,fav_cuisine,fav_cuisine_coded,fav_food,food_childhood,fries,fruit_day,grade_level,greek_food,healthy_feeling,healthy_meal,ideal_diet,ideal_diet_coded,income,indian_food,italian_food,...,3_duplicated_812,2_duplicated_1311,4_duplicated_567,Journalist_duplicated_0,HISPANIC CUISINE.,2_duplicated_1312,1_duplicated_1541,"rice, beans, and chicken / pizza/ tenders",1_duplicated_1542,3_duplicated_813,3_duplicated_814,2_duplicated_1313,3_duplicated_815,"a cup of rice, vegetables, and meat.","Being able to balance between sweets, vegetables, fruits, carbohydrates, and fat.",3_duplicated_816,5_duplicated_628,2_duplicated_1314,3_duplicated_817,5_duplicated_629,2_duplicated_1315,"Vegetables, Meat, and rice.",3_duplicated_818,House-wife,5_duplicated_630,1_duplicated_1543,3_duplicated_819,3_duplicated_820,2_duplicated_1316,3_duplicated_821,1_duplicated_1544,2_duplicated_1317,2_duplicated_1318,725_duplicated_20,345_duplicated_24,nan_duplicated_118,4_duplicated_568,2_duplicated_1319,575_duplicated_1,135


### **Getting The Dimensions**

In [13]:
from pandas import DataFrame
class test(DataFrame):
    def get_dims(self):
        print(f"Cols:{self.shape[1]}")
        print(f"Rows:{self.shape[0]}")
    def modify_col_case(self, option=""):
        if option == "title":                             
            self = self.rename(columns=lambda x: x[0].title() + x[1:]).head()
        elif option =="upper":
            self = self.rename(columns=lambda x: x[0].upper() + x[1:]).head()
        else:
            self = self.rename(columns=lambda x: x[0].lower() + x[1:]).head()
df = test(df)

Calling the Function

In [14]:
df.get_dims()

Cols:61
Rows:125


In [15]:
df.modify_col_case(option="title")

In [44]:
import pandas
df.rename(columns=lambda x: x[0].lower() + x[1:]).head()

,gPA,gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,cook,comfort_food_reasons_coded.1,cuisine,diet_current,diet_current_coded,drink,eating_changes,eating_changes_coded,eating_changes_coded1,eating_out,employment,ethnic_food,exercise,father_education,father_profession,fav_cuisine,fav_cuisine_coded,fav_food,food_childhood,fries,fruit_day,grade_level,greek_food,healthy_feeling,healthy_meal,ideal_diet,ideal_diet_coded,income,indian_food,italian_food,life_rewarding,marital_status,meals_dinner_friend,mother_education,mother_profession,nutritional_check,on_off_campus,parents_cook,pay_meal_out,persian_food,self_perception_weight,soup,sports,thai_food,tortilla_calories,turkey_calories,type_sports,veggies_day,vitamins,waffle_calories,weight
0,2.4,2,1,430,NaN,315.0,1,none,we dont have comfort,9.0,2.0,9,NaN,eat good and exercise,1,1.0,eat faster,1,1,3,3.0,1,1.0,5.0,profesor,Arabic cuisine,3,1.0,rice and chicken,2,5,2,5,2,looks not oily,being healthy,8,5.0,5,5,1.0,1.0,"rice, chicken, soup",1.0,unemployed,5,1.0,1,2,5.0,3.0,1.0,1.0,1,1165.0,345,car racing,5,1,1315,187
1,3.654,1,1,610,3.0,420.0,2,"chocolate, chips, ice cream","Stress, bored, anger",1.0,3.0,1,1.0,I eat about three times a day with some snacks. I try to eat healthy but it doesn't always work out that- sometimes eat fast food and mainly eat at Laker/ Egan,2,2.0,I eat out more than usual.,1,2,2,2.0,4,1.0,2.0,Self employed,Italian,1,1.0,"chicken and biscuits, beef soup, baked beans",1,4,4,4,5,"Grains, Veggies, (more of grains and veggies), small protein and fruit with dairy","Try to eat 5-6 small meals a day. While trying to properly distribute carbs, protein, fruits, veggies, and dairy.",3,4.0,4,4,1.0,2.0,"Pasta, steak, chicken",4.0,Nurse RN,4,1.0,1,4,4.0,3.0,1.0,1.0,2,725.0,690,Basketball,4,2,900,155
2,3.3,1,1,720,4.0,420.0,2,"frozen yogurt, pizza, fast food","stress, sadness",1.0,1.0,1,3.0,"toast and fruit for breakfast, salad for lunch, usually grilled chicken and veggies (or some variation) for dinner",3,1.0,sometimes choosing to eat fast food instead of cooking simply for convenience,1,3,2,3.0,5,2.0,2.0,owns business,italian,1,3.0,"mac and cheese, pizza, tacos",1,5,3,5,6,usually includes natural ingredients; nonprocessed food,i would say my ideal diet is my current diet,6,6.0,5,5,7.0,2.0,"chicken and rice with veggies, pasta, some kind of healthy recipe",2.0,owns business,4,2.0,1,3,5.0,6.0,1.0,2.0,5,1165.0,500,none,5,1,900,I'm not answering this.
3,3.2,1,1,430,3.0,420.0,2,"Pizza, Mac and cheese, ice cream",Boredom,2.0,2.0,2,2.0,"College diet, cheap and easy foods most nights. Weekends traditionally, cook better homemade meals",2,2.0,Accepting cheap and premade/store bought foods,1,3,2,3.0,5,3.0,2.0,Mechanic,Turkish,3,1.0,"Beef stroganoff, tacos, pizza",2,4,4,5,7,"Fresh fruits& vegetables, organic meats","Healthy, fresh veggies/fruits & organic foods",2,6.0,5,5,2.0,2.0,Grilled chicken \rStuffed Shells\rHomemade Chili,4.0,Special Education Teacher,2,1.0,1,2,5.0,5.0,1.0,2.0,5,725.0,690,NaN,3,1,1315,"Not sure, 240"
4,3.5,1,1,720,2.0,420.0,2,"Ice cream, chocolate, chips","Stress, boredom, cravings",1.0,1.0,1,2.0,I try to eat healthy but often struggle because of living on campus. I still try to keep the choices I do make balanced with fruits and vegetables and limit the sweats.,2,2.0,I have eaten generally the same foods but I do find myself eating the same food frequently due to what I have found I like from egan and the laker.,3,4,2,2.0,4,1.0,4.0,IT,Italian,1,3.0,"Pasta, chicken tender, pizza",1,4,4,4,6,"A lean protein such as grilled chicken, green vegetables and brown rice or other whole grain",Ideally I would like to be able to eat healthier foods in order to loose weight.,2,6.0,2,5,1.0,1.0,"Chicken Parmesan, Pulled Pork, Spaghetti and meatballs",5.0,Substance Abuse Conselor,3,1.0,1,4,2.0,4.0,1.0,1.0,4,940.0,500,Softball,4,2,760,190


### Drop Columns